In [210]:
import kfp

from typing import NamedTuple
import kfp.components as comp
from kfp import compiler, dsl
from kfp import dsl
from kfp.components import InputPath, OutputPath
from kubernetes.client.models import V1EnvVar

# SCHEME

In [211]:
def ObtainSchemeIsland(pilot_name, domain_, url_) -> str:

    import requests, json

    if pilot_name == "Virtual":
        if domain_ == "heat":
            return '[{"id":1,"name":"energy_island1","type":{"id":4,"name":"energy_island","label":"Energy island","category":"structural","renovable":None},"label":"Energy Island 1","description":None,"geo_location":None,"parent":None,"child":[{"id":2,"name":"building1","type":"Building","label":"Building 1","description":None,"geo_location":None},{"id":3,"name":"building2","type":"Building","label":"Building 2","description":None,"geo_location":None}],"measurements":[],"owner":None},{"id":4,"name":"user1","type":{"id":18,"name":"user","label":"User","category":"user","renovable":None},"label":"User 1","description":None,"geo_location":None,"parent":None,"child":[],"measurements":[],"owner":None},{"id":5,"name":"user2","type":{"id":18,"name":"user","label":"User","category":"user","renovable":None},"label":"User 2","description":None,"geo_location":None,"parent":None,"child":[],"measurements":[],"owner":None},{"id":2,"name":"building1","type":{"id":3,"name":"building","label":"Building","category":"structural","renovable":None},"label":"Building 1","description":None,"geo_location":None,"parent":{"id":1,"name":"energy_island1","type":"Energy island","label":"Energy Island 1","description":None,"geo_location":None},"child":[{"id":6,"name":"gas_boiler1","type":"Gas boiler","label":"Gas Boiler 1","description":None,"geo_location":None},{"id":21,"name":"front_test","type":"Ev charging station","label":"Front test","description":None,"geo_location":None}],"measurements":[{"id":10,"name":"heat_consumed","label":"Heat Consumed","description":None,"icon":None,"type":{"id":7,"name":"power_mw","label":"Power [MW]","unit":"MW","factor":1000000.0,"color":"#4CAF50","base_unit":"W","metric_type":"electricity"},"direction":"out"},{"id":11,"name":"current_temperature","label":"Current temperature","description":None,"icon":None,"type":{"id":4,"name":"temperature_c","label":"Temperature[°C]","unit":"°C","factor":1.0,"color":"#4CAF50","base_unit":"°C","metric_type":"temperature"},"direction":None}],"owner":4},{"id":3,"name":"building2","type":{"id":3,"name":"building","label":"Building","category":"structural","renovable":None},"label":"Building 2","description":None,"geo_location":None,"parent":{"id":1,"name":"energy_island1","type":"Energy island","label":"Energy Island 1","description":None,"geo_location":None},"child":[{"id":7,"name":"gas_boiler2","type":"Gas boiler","label":"Gas Boiler 2","description":None,"geo_location":None},{"id":8,"name":"solar_collector1","type":"Solar thermal collector","label":"Solar Collector 1","description":None,"geo_location":None}],"measurements":[{"id":10,"name":"heat_consumed","label":"Heat Consumed","description":None,"icon":None,"type":{"id":7,"name":"power_mw","label":"Power [MW]","unit":"MW","factor":1000000.0,"color":"#4CAF50","base_unit":"W","metric_type":"electricity"},"direction":"out"},{"id":11,"name":"current_temperature","label":"Current temperature","description":None,"icon":None,"type":{"id":4,"name":"temperature_c","label":"Temperature[°C]","unit":"°C","factor":1.0,"color":"#4CAF50","base_unit":"°C","metric_type":"temperature"},"direction":None}],"owner":5},{"id":6,"name":"gas_boiler1","type":{"id":7,"name":"gas_boiler","label":"Gas boiler","category":"energy","renovable":0},"label":"Gas Boiler 1","description":None,"geo_location":None,"parent":{"id":2,"name":"building1","type":"Building","label":"Building 1","description":None,"geo_location":None},"child":[],"measurements":[{"id":9,"name":"heat_supply","label":"Heat Supply","description":None,"icon":None,"type":{"id":7,"name":"power_mw","label":"Power [MW]","unit":"MW","factor":1000000.0,"color":"#4CAF50","base_unit":"W","metric_type":"electricity"},"direction":"in"}],"owner":4},{"id":7,"name":"gas_boiler2","type":{"id":7,"name":"gas_boiler","label":"Gas boiler","category":"energy","renovable":0},"label":"Gas Boiler 2","description":None,"geo_location":None,"parent":{"id":3,"name":"building2","type":"Building","label":"Building 2","description":None,"geo_location":None},"child":[],"measurements":[{"id":9,"name":"heat_supply","label":"Heat Supply","description":None,"icon":None,"type":{"id":7,"name":"power_mw","label":"Power [MW]","unit":"MW","factor":1000000.0,"color":"#4CAF50","base_unit":"W","metric_type":"electricity"},"direction":"in"}],"owner":5},{"id":8,"name":"solar_collector1","type":{"id":13,"name":"solar_thermal_collector","label":"Solar thermal collector","category":"energy","renovable":100},"label":"Solar Collector 1","description":None,"geo_location":None,"parent":{"id":3,"name":"building2","type":"Building","label":"Building 2","description":None,"geo_location":None},"child":[],"measurements":[{"id":9,"name":"heat_supply","label":"Heat Supply","description":None,"icon":None,"type":{"id":7,"name":"power_mw","label":"Power [MW]","unit":"MW","factor":1000000.0,"color":"#4CAF50","base_unit":"W","metric_type":"electricity"},"direction":"in"}],"owner":5},{"id":21,"name":"front_test","type":{"id":19,"name":"ev_charging_station","label":"Ev charging station","category":"energy","renovable":100},"label":"Front test","description":None,"geo_location":None,"parent":{"id":2,"name":"building1","type":"Building","label":"Building 1","description":None,"geo_location":None},"child":[],"measurements":[],"owner":4}]'
        elif domain_ == "electricity":
            return '[{"name": "building1", "type": {"name": "building"}, "child": [{"name": "cogenerator_1"}, {"name": "wind_farm_1"}]}, {"name": "building2", "type": {"name": "building"}, "child": [{"name": "cogenerator_2"}, {"name": "wind_farm_1"}, {"name": "pv_panel_1"}]}]'
    else:
        url = "http://api-swagger-ren-prototype.apps.paas-dev.psnc.pl/api/assets"
        r = requests.get(url)
        asset_dict = r.json()
        return json.dumps(asset_dict)

# RENEWABILITY ASSETS

In [306]:
def GetRenewability_Assets(url_pilot, domain_) -> str:

    import json

    if url_pilot == "Virtual":
        if domain_ == "heat":
            renewability_score =  {
                "solar_collector1": 1,
                "gas_boiler1": 0.2,
                "gas_boiler2": 0.3
            }
        elif domain_ == "electricity":
            renewability_score =  {
                "pv_panel_1": 1,
                "wind_farm_1": 1,
                "cogenerator_1": 0.2,
                "cogenerator_2": 0.2
            }
        else:
            renewability_score = {}
    else:
        renewability_score = {}
    
    return json.dumps(renewability_score)

# COMPOSE GRID GRAPH

In [307]:
def GetGridInfo(renewability_score:str, dict_str:str, output_graph_path: OutputPath(str)):
    
    import requests, json
    import networkx as nx
    from networkx.readwrite import json_graph
    
    dicts = json.loads(dict_str)
    print(len(dicts))

    renewability_score = json.loads(renewability_score)
    print(renewability_score.keys())

    def GetGraph(dicts, renewability_score):
        G = nx.Graph()
        i = 1
        j = 1
        for dict_asset in dicts:
            try:
                if dict_asset["type"]["name"] == "building":
                    G.add_node(dict_asset["name"])
                    G.nodes[dict_asset["name"]]["type"] = "consumer"
                    G.nodes[dict_asset["name"]]["pos"] = [2,i]
                    i = i + 1
                    for child_asset in dict_asset["child"]:
                        if child_asset["name"] != "front_test":
                            if child_asset["name"] not in list(G.nodes):
                                G.add_node(child_asset["name"])
                                G.nodes[child_asset["name"]]["type"] = "supplier"
                                G.nodes[child_asset["name"]]["pos"] = [1,j]
                                j = j + 1
                            G.nodes[child_asset["name"]]["ren_score"] = renewability_score[child_asset["name"]]
                            G.add_edge(child_asset["name"], dict_asset["name"])
            except IndexError:
                pass
        return G
    
    G = GetGraph(dicts, renewability_score)
    data1 = json_graph.node_link_data(G)
    
    with open(output_graph_path, "w") as file:
        json.dump(data1, file)


# OBTAIN DATA

In [308]:
def GetData(measurement_name: str, min_date: str, max_date: str,url_pilot : str,type_measurement :str,key_measurement : str,
            filter_vars:list , filter_cases: list, output_data_forecast: OutputPath(str)):

    import requests # To REQUIREMENTS
    import json
    import pandas as pd # To REQUIREMENTS
    import maya # To REQUIREMENTS
    from tqdm import tqdm
    from icecream import ic
    from discord_webhook import DiscordWebhook
    from retry import retry # TO REQUIREMENTS

    #Functions definitions

    def GetRequest(url, headers ={}, payload = {}):

        from urllib3.exceptions import InsecureRequestWarning
        import warnings
        import contextlib

        old_merge_environment_settings = requests.Session.merge_environment_settings

        @contextlib.contextmanager
        def no_ssl_verification():
            opened_adapters = set()

            def merge_environment_settings(self, url, proxies, stream, verify, cert):
                # Verification happens only once per connection so we need to close
                # all the opened adapters once we're done. Otherwise, the effects of
                # verify=False persist beyond the end of this context manager.
                opened_adapters.add(self.get_adapter(url))

                settings = old_merge_environment_settings(self, url, proxies, stream, verify, cert)
                settings['verify'] = False

                return settings

            requests.Session.merge_environment_settings = merge_environment_settings

            try:
                with warnings.catch_warnings():
                    warnings.simplefilter('ignore', InsecureRequestWarning)
                    yield
            finally:
                requests.Session.merge_environment_settings = old_merge_environment_settings

                for adapter in opened_adapters:
                    try:
                        adapter.close()
                    except:
                        pass
        
        with no_ssl_verification():
            response = requests.request("GET", url, headers = headers, data = payload)
            
        try:
            return response.json()
        except:
            dict_ = {
                "status_code": response.status_code,
                "text": response.text
            }
            return dict_
    def DownloadAssetsData(measurement_name, url_pilot,bucket = "renergetic", min_date = "yesterday", max_date = "tomorrow"):
        
        from datetime import datetime
        import pandas as pd
        import maya
        from tqdm import tqdm
        from icecream import ic

        test = True

        try:
            min_date_from = maya.when(min_date).datetime()
        except:
            ValueError("Please introduce correct time format for MIN_DATE")
        
        try: 
            max_date_from = maya.when(max_date).datetime()
        except:
            ValueError("Please introduce correct time format for MAX_DATE")
        
        datelist = pd.date_range(min_date_from, max_date_from)

        data_ = []
        for i in tqdm(range(len(datelist)-1)):
            from_obj = datelist[i]
            to_obj = datelist[i+1]
            from_ = datetime.strftime(from_obj, "%Y-%m-%d 00:00:00")
            to_ = datetime.strftime(to_obj, "%Y-%m-%d 00:00:00")

            url = url_pilot + "/data?measurements={measurement_name}&from={from_}&to={to_}"\
                .format(measurement_name = measurement_name, from_ = from_, to_= to_)
            info_ = GetRequest(url)
            if type(info_) == list:
                data_ = data_ + info_
            elif type(info_) == dict:
                print("Error")
                print(from_)
                print(to_)
                print(info_)
                print(url)
        return data_
    def DataFrameAssests(list_data, name_field):
        dicts = []
        for data in list_data:
            try:
                if "energy" in data["fields"].keys():
                    name_value = "energy"
                else:
                    name_value = name_field
                
                if name_value not in data["fields"].keys():
                    print(data)
                dict_ = {
                    "asset_name": data["tags"]["asset_name"],
                    "value": float(data["fields"][name_value]),
                    "ds": data["fields"]["time"]
                }

                if "type_data" in data["tags"].keys():
                    dict_["type"] = data["tags"]["type_data"]
                elif "typeData" in data["tags"].keys():
                    dict_["type"] = data["tags"]["typeData"]
                else:
                    dict_["type"] = "None"

                if "measurement_type" in data["tags"].keys():
                    dict_["measurement_type"] = data["tags"]["measurement_type"]
                else:
                    dict_["measurement_type"] = "None"
                
                if "direction" in data["tags"].keys():
                    dict_["direction"] = data["tags"]["direction"]
                else:
                    dict_["direction"] = "None"
                if "domain" in data["tags"].keys():
                    dict_["domain"] = data["tags"]["domain"]
                else:
                    dict_["domain"] = "None"
                
                if "sensor_id" in data["tags"].keys():
                    dict_["id_sensor"] = data["tags"]["sensor_id"]
                else:
                    dict_["id_sensor"] = "None"
                
                if "interpolation_method" in data["tags"].keys():
                    dict_["interpolation"] = data["tags"]["interpolation_method"]
                else:
                    dict_["interpolation"] = "None"

                dicts.append(dict_)
            except KeyError:
                continue
        return pd.DataFrame(dicts)
    
    @retry(tries= 3)
    def DownloadAndProcess(measurement_name, url_pilot, min_date, max_date, key_measurement):
        # max_date = maya.now().add(days = 3).iso8601()
        list_ = DownloadAssetsData(measurement_name, url_pilot,min_date = min_date, max_date = max_date)
        data = DataFrameAssests(list_, key_measurement)

        return data
    
    def FilterCases(var_value, filter_cases):
        if var_value in filter_cases:
            return True
        else:
            return False

    def FilterData(data, type_measurement, filter_vars, filter_cases):

        try:
            data = data[data.type == type_measurement]
        except:
            pass

        for i in range(len(filter_vars)):
            data["Filter"] = data[filter_vars[i]].apply(FilterCases, filter_cases = filter_cases[i])
            data = data[data["Filter"] == True]
        
        return data    
    
    def SendAlert(data):
        ic(data.shape[0])

        if data.shape[0] == 0:
            url_disc = "https://discord.com/api/webhooks/1002537248622923816/_9XY9Hi_mjzh2LTVqnmSKXlIFJ5rgBO2b8xna5pynUrzALgtC4aXSFq89uMdlW_v-ZzT"
            message = "Not enough data for {measurement_name}".format(measurement_name = measurement_name)
            webhook = DiscordWebhook(url = url_disc, content = message)
            webhook.execute()
            
            raise ValueError("Void data to forecast")

    # Code Execution
    
    data = DownloadAndProcess(measurement_name, url_pilot, min_date, max_date, key_measurement)
    
    data = FilterData(data, type_measurement, filter_vars, filter_cases)
    
    SendAlert(data)
    

    data_output = {
        "value": data["value"].tolist(),
        "time_registered": data["ds"].tolist(),
        "asset_name": data["asset_name"].tolist(),
        "domain": data["domain"].tolist(),
        "direction": data["direction"].tolist()
    }

    with open(output_data_forecast, "w") as file:
        json.dump(data_output, file)

In [309]:
def Computation(input_data_path: InputPath(str), input_graph_path:InputPath(str), domain, output_data_path: OutputPath(str)):

    import json
    from networkx.readwrite import json_graph
    from tqdm import tqdm
    import pandas as pd

    with open(input_data_path) as file:
        data = json.load(file)

    with open(input_graph_path) as file:
        data_graph = json.load(file)
    
    data = pd.DataFrame(data)
    data = data[data.domain == domain]
    G = json_graph.node_link.node_link_graph(data_graph)

    def CalculateRenewabilityScore(data_heat, G):
        import networkx as nx
        import numpy as np
        from icecream import ic
        from tqdm import tqdm 
        import warnings
        warnings.filterwarnings("ignore")

        data_heat = data_heat[["asset_name", "value", "time_registered", "direction"]]

        data_consumed = data_heat[data_heat.direction == "in"]
        data_consumed = data_consumed.drop(["direction"], axis = 1)
        data_consumed.columns = ["asset_name", "heat_consumption", "ds"]
        
        data_supply = data_heat[data_heat.direction == "out"]
        data_supply = data_supply.drop(["direction"], axis = 1)
        data_supply.columns = ["asset_name", "heat_supply", "ds"]
        
        
        
        for node in list(G.nodes):
            if G.nodes[node]["type"] == "consumer":
                value_node = data_consumed[data_consumed.asset_name == node].heat_consumption.tolist()[-1]
                G.nodes[node]["weight"] = value_node
            elif G.nodes[node]["type"] == "supplier":
                node_data = node
                value_node = data_supply[data_supply.asset_name == node_data].heat_supply.tolist()[-1]
                G.nodes[node]["weight"] = value_node
        SGs = [G.subgraph(c).copy() for c in nx.connected_components(G)]

        i = 1

        G_final = nx.Graph()
        for SG in SGs:
            energy_supply = 0
            energy_consumed = 0
            for node in list(SG.nodes):
                if SG.nodes[node]["type"] == "consumer":
                    energy_consumed = energy_consumed + SG.nodes[node]["weight"]
                elif SG.nodes[node]["type"] == "supplier":
                    energy_supply = energy_supply + SG.nodes[node]["weight"]
            SG.add_node("Grid")
            SG.nodes["Grid"]["type"] = "grid_element"
            SG.nodes["Grid"]["weight"] = 0
            SG.nodes["Grid"]["ren_score"] = 0.25
            if energy_supply >= energy_consumed:
                SG.nodes["Grid"]["weight"] = energy_supply - energy_consumed 
                for node in list(SG.nodes):
                    if SG.nodes[node]["type"] == "supplier":
                        SG.add_edge("Grid",node)
            else:
                SG.nodes["Grid"]["weight"] = energy_consumed - energy_supply
                for node in list(SG.nodes):
                    if SG.nodes[node]["type"] == "consumer":
                        SG.add_edge("Grid", node)
            i = i+1
            I_g = nx.incidence_matrix(SG)
            I_g = np.matrix(I_g.todense())
            ap_mat = np.zeros((I_g.shape[0],1))
            ap_mat[I_g.shape[0]-1,0] = -1
            LM = np.matrix(np.hstack((I_g,ap_mat)))


            result_vector = np.zeros((I_g.shape[0],1))

            for i in range(SG.number_of_nodes()-1):
                name_node = list(SG.nodes)[i]
                result_vector[i,0] = SG.nodes[name_node]["weight"]

            weight_edges = LM.getI()*result_vector



            j = 0
            for edge in SG.edges:
                nx.set_edge_attributes(SG,{(edge[0], edge[1]): {"weight": weight_edges[j]}})
                j = j + 1
            G_final = nx.compose(G_final, SG)

        dict_ = {}

        for node in list(G_final.nodes):
            print(node)
            print(G_final.nodes[node]["type"])
            if G_final.nodes[node]["type"] == "consumer":
                consumption = 0
                for edge in list(G_final.edges(node)):
                    consumption_edge = G_final.edges[edge]["weight"]
                    r_score = G_final.nodes[edge[1]]["ren_score"]
                    consumption = consumption + consumption_edge * r_score
                dict_[node] = np.round(consumption / G_final.nodes[node]["weight"]*100,2)[0][0]
        
        dict_["ds"] = data_consumed["ds"].tolist()[-1]  

        return dict_ , G_final
    
    list_dicts = []
    
    for ds_unique in tqdm(pd.unique(data.time_registered)):
        data_ds = data[data.time_registered == ds_unique]
        a,b = CalculateRenewabilityScore(data_ds, G)
        list_dicts.append(a)

    with open(output_data_path, "w") as file:
        json.dump(list_dicts, file)


In [349]:
def SendRenewabilityScores(input_ren_path: InputPath(str), domain_, url_pilot):

    import json
    from datetime import datetime
    import maya
    import requests
    from discord_webhook import DiscordWebhook

    with open(input_ren_path) as file:
        dicts_ren = json.load(file)

    def SendData(url_pilot, data_post, asset_name):
        url = url_pilot
        print(url)
        headers = {
            "Content-Type": "application/json"
        }

        try:
            response = requests.request("POST", url, headers=headers, data=json.dumps(data_post))
            status_code = response.status_code

        except:
            url_disc = "https://discord.com/api/webhooks/1002537248622923816/_9XY9Hi_mjzh2LTVqnmSKXlIFJ5rgBO2b8xna5pynUrzALgtC4aXSFq89uMdlW_v-ZzT"
            message = "Error in updating value for measurement name: {measurement_name} in asset: {asset_name} in time {time_pred}"\
                .format(measurement_name = "RENEWABILITY", asset_name = asset_name, time_pred = data_post["fields"]["time"])
            webhook = DiscordWebhook(url = url_disc, content = message)
            webhook.execute()
            status_code = 200
        
        if status_code > 299:
            url_disc = "https://discord.com/api/webhooks/1002537248622923816/_9XY9Hi_mjzh2LTVqnmSKXlIFJ5rgBO2b8xna5pynUrzALgtC4aXSFq89uMdlW_v-ZzT"
            message = "Error in sending the value for measurement name: {measurement_name} in asset: {asset_name} in time {time_pred}"\
                .format(measurement_name = "RENEWABILITY", asset_name = asset_name, time_pred = data_post["fields"]["time"])
            print(response.text)
            print(json.dumps(data_post))
            webhook = DiscordWebhook(url = url_disc, content = message)
            webhook.execute()

    data_post = {
                "bucket": "renergetic",
                "measurement": "renewability",
                "fields":{
                    "percentage": None,
                    "time": None,
                },
                "tags":{
                    "domain": domain_,
                    "typeData": "forecasting",
                    "direction": None,
                    "prediction_window": "24h",
                    "asset_name": None,
                    "time_prediction": datetime.strftime(maya.now().datetime(), "%Y-%m-%d %H:%M:%S")
                }
            }

    for dict_ in dicts_ren:
        data_post["fields"]["time"] = dict_["ds"]

        for key_ in dict_.keys():
            if key_ != "ds":
                data_post["fields"]["percentage"] = dict_[key_]
                data_post["tags"]["asset_name"] = key_
                SendData(url_pilot, data_post, key_)




In [350]:
def GetThresholds(output_threshold_path: OutputPath(str)):
    import json

    dict_ = {
        "building1": 40,
        "building2": 20
    }

    with open(output_threshold_path, "w") as file:
        json.dump(dict_, file)

In [351]:
def GetDemandsDefinition(actionType: str) -> str:  
    # GET INFORMATION FROM DEMANDS DEFINITION
    import json
    import requests
    
    url = "http://api-ren-prototype.apps.paas-dev.psnc.pl/api/demandRequests/definition"
    payload={}
    headers = {}

    response = requests.request("GET", url, headers=headers, data=payload)
    dict_demands = {}
    for demand_response in response.json():
        if demand_response["action_type"] == actionType:
                dict_demands = {
                    "id": demand_response["id"],
                    "action": demand_response["action"],
                    "message": demand_response["message"],
                    "ext": None,
                    "action_type": "DECREASE",
                    "information_tile_id": None
                }
    
    if dict_demands == {}:
        raise ValueError("Demands definition empty")
    else:
        return json.dumps(dict_demands)

In [352]:
def GetAssetProperties()-> str:
    import requests
    import json
    
    #GET INFORMATION FROM ASSETS
    
    url_assets = "http://api-ren-prototype.apps.paas-dev.psnc.pl/api/assets/"

    payload={}
    headers = {}

    response = requests.request("GET", url_assets, headers=headers, data=payload)
    
    dict_assets = {"id_asset": [], "name_asset": []}
    for item in response.json(): 
        if item["type"]["name"] == "building":
            dict_assets["id_asset"] = dict_assets["id_asset"] + [item["id"]]
            dict_assets["name_asset"] = dict_assets["name_asset"] + [item["name"]]
    
    return json.dumps(dict_assets)

In [353]:
def SendRenewabilityRecommendation(input_ren_path: InputPath(str), input_thresholds_path: InputPath(str), input_demand_definition, input_dict_assets):

    import json
    import maya
    import pandas as pd
    import requests

    with open(input_ren_path) as file:
        dicts_ren = json.load(file)
    
    with open(input_thresholds_path) as file:
        thresholds = json.load(file)

    demand_definition = json.loads(input_demand_definition)
    dict_assets = pd.DataFrame(json.loads(input_dict_assets))
    
    for dict_ in dicts_ren:
        for key_ in dict_.keys():
            if key_ in thresholds.keys():
                if dict_[key_] < thresholds[key_]:
                    url = "http://api-swagger-ren-prototype.apps.paas-dev.psnc.pl/api/demandRequests"
    
                    t_now = maya.now()
                    t_end = t_now.add(hours = 1)
                    
                    id_asset = dict_assets[dict_assets.name_asset == key_].id_asset.tolist()[0]

                    dict_to = {
                        "asset_id": id_asset,
                        "demand_definition": demand_definition,
                        "demand_start": t_now.epoch,
                        "demand_stop": t_end.epoch,
                        "demand_update": None
                    }
                    
                    headers = {
                            "Content-Type": "application/json"
                        }
                    
                    response = requests.request("POST", url, headers=headers, data=json.dumps(dict_to))
                    
                    if response.status_code != 200:
                        print(response.status_code)
                        print(response.text)
                        print(dict_to)
                        raise ValueError
                else:
                    print("NOT REN Alert at {ds}".format(ds = dict_["ds"]))


# DEFINE PIPELINE

In [354]:
def PipelineRenewability(
        pilot_name,
        domain_,
        measurement,
        min_date,
        max_date,
        url_pilot,
        type_measurement,
        key_measurement,
        filter_vars,
        filter_case):
    
    env_var = V1EnvVar(name='HOME', value='/tmp')
    
    get_scheme_op = comp.create_component_from_func(ObtainSchemeIsland, packages_to_install=["requests"],output_component_file="scheme_island_comp.yaml")
    get_scores_assets_op = comp.create_component_from_func(GetRenewability_Assets, output_component_file="renewability_assets_comp.yaml")
    get_renewability_thresholds = comp.create_component_from_func(GetThresholds, output_component_file= "get_thresholds_comp.yaml")
    get_grid_info_op = comp.create_component_from_func(GetGridInfo, output_component_file="get_grid_info_comp.yaml", packages_to_install= ["networkx", "requests"])
    download_data_op = comp.create_component_from_func(
        GetData, packages_to_install = ["requests", "numpy", "maya","pandas", "icecream", "tqdm", "discord-webhook", "retry", "urllib3"],
        output_component_file = "download_data_op_component.yaml")
    computation_ren_op = comp.create_component_from_func(
        Computation, packages_to_install= ["pandas", "networkx", "tqdm", "icecream", "numpy", "scipy"], output_component_file= "computation_ren_comp.yaml"
    )

    send_ren_op = comp.create_component_from_func(SendRenewabilityScores,packages_to_install= ["maya", "requests","discord_webhook"], output_component_file= "send_scores_comp.yaml")
    get_demand_definition_op = comp.create_component_from_func(GetDemandsDefinition, packages_to_install=["requests"], output_component_file= "get_definition_comp.yaml")
    get_asset_info_op = comp.create_component_from_func(GetAssetProperties, packages_to_install= ["requests"], output_component_file= "get_asset_info_comp.yaml")
    send_recommendation_op = comp.create_component_from_func(SendRenewabilityRecommendation, packages_to_install=["maya", "requests","pandas"],output_component_file="send_recommendation.yaml")

    download_task = (download_data_op(measurement, min_date, max_date, url_pilot,type_measurement, key_measurement, filter_vars, filter_case).add_env_variable(env_var)
                            .set_memory_request('500M')
                            .set_memory_limit('1Gi')
                            .set_cpu_request('1')
                            .set_cpu_limit('2'))

    scheme_task = get_scheme_op(pilot_name, domain_, url_pilot)
    scores_task = get_scores_assets_op(pilot_name, domain_)
    grid_task = get_grid_info_op(scores_task.output,scheme_task.output)
    
    computation_task = computation_ren_op(download_task.output, grid_task.output, domain_)

    

    


    send_ren_task = send_ren_op(computation_task.output, domain_, url_pilot)

    get_thresholds_task = get_renewability_thresholds()
    get_demand_definition_task = get_demand_definition_op("DECREASE")
    get_asset_info_task = get_asset_info_op()
    send_recommendation_task = send_recommendation_op(computation_task.output, get_thresholds_task.output, get_demand_definition_task.output, get_asset_info_task.output)

compiler.Compiler().compile(pipeline_func = PipelineRenewability, package_path ="Renewability_Pipeline.yaml")

# TESTS

In [220]:
import requests
url = "http://api-swagger-ren-prototype.apps.paas-dev.psnc.pl/api/assets"
r = requests.get(url)

KeyboardInterrupt: 

In [ ]:
import json
with open("asset_list.json", "w") as file:
    json.dump(r.json(), file)

In [ ]:
asset_dict = r.json()

In [ ]:
for asset in asset_dict:
    if asset["name"] == "gas_boiler1":
        print(asset)

{'id': 14, 'name': 'gas_boiler1', 'type': {'id': 7, 'name': 'gas_boiler', 'label': 'Gas boiler', 'renovable': 0}, 'label': 'Gas Boiler 1', 'geo_location': None, 'parent': None, 'child': [], 'measurements': [], 'user': None, 'asset_category': None, 'details': {}}


In [ ]:
asset_str = json.dumps(asset_dict)

In [ ]:
def CalculateRenewabilityScore(data_heat, G):
    import networkx as nx
    import numpy as np
    from icecream import ic
    from tqdm import tqdm 
    import warnings
    warnings.filterwarnings("ignore")

    data_heat = data_heat[["asset_name", "value", "ds", "direction"]]

    data_consumed = data_heat[data_heat.direction == "in"]
    data_consumed = data_consumed.drop(["direction"], axis = 1)
    data_consumed.columns = ["asset_name", "heat_consumption", "ds"]
    
    data_supply = data_heat[data_heat.direction == "out"]
    data_supply = data_supply.drop(["direction"], axis = 1)
    data_supply.columns = ["asset_name", "heat_supply", "ds"]
    
    
    
    for node in list(G.nodes):
        if G.nodes[node]["type"] == "consumer":
            value_node = data_consumed[data_consumed.asset_name == node].heat_consumption.tolist()[-1]
            G.nodes[node]["weight"] = value_node
        elif G.nodes[node]["type"] == "supplier":
            node_data = node
            value_node = data_supply[data_supply.asset_name == node_data].heat_supply.tolist()[-1]
            G.nodes[node]["weight"] = value_node
    SGs = [G.subgraph(c).copy() for c in nx.connected_components(G)]

    i = 1

    G_final = nx.Graph()
    for SG in SGs:
        energy_supply = 0
        energy_consumed = 0
        for node in list(SG.nodes):
            if SG.nodes[node]["type"] == "consumer":
                energy_consumed = energy_consumed + SG.nodes[node]["weight"]
            elif SG.nodes[node]["type"] == "supplier":
                energy_supply = energy_supply + SG.nodes[node]["weight"]
        SG.add_node("Grid")
        SG.nodes["Grid"]["type"] = "grid_element"
        SG.nodes["Grid"]["weight"] = 0
        SG.nodes["Grid"]["ren_score"] = 0.25
        if energy_supply >= energy_consumed:
            SG.nodes["Grid"]["weight"] = energy_supply - energy_consumed 
            for node in list(SG.nodes):
                if SG.nodes[node]["type"] == "supplier":
                    SG.add_edge("Grid",node)
        else:
            SG.nodes["Grid"]["weight"] = energy_consumed - energy_supply
            for node in list(SG.nodes):
                if SG.nodes[node]["type"] == "consumer":
                    SG.add_edge("Grid", node)
        i = i+1
        I_g = nx.incidence_matrix(SG)
        I_g = np.matrix(I_g.todense())
        ap_mat = np.zeros((I_g.shape[0],1))
        ap_mat[I_g.shape[0]-1,0] = -1
        LM = np.matrix(np.hstack((I_g,ap_mat)))


        result_vector = np.zeros((I_g.shape[0],1))

        for i in range(SG.number_of_nodes()-1):
            name_node = list(SG.nodes)[i]
            result_vector[i,0] = SG.nodes[name_node]["weight"]

        weight_edges = LM.getI()*result_vector



        j = 0
        for edge in SG.edges:
            nx.set_edge_attributes(SG,{(edge[0], edge[1]): {"weight": weight_edges[j]}})
            j = j + 1
        G_final = nx.compose(G_final, SG)

    dict_ = {}

    for node in list(G_final.nodes):
        if G_final.nodes[node]["type"] == "consumer":
            consumption = 0
            for edge in list(G_final.edges(node)):
                consumption_edge = G_final.edges[edge]["weight"]
                r_score = G_final.nodes[edge[1]]["ren_score"]
                consumption = consumption + consumption_edge * r_score
            dict_[node] = np.round(consumption / G_final.nodes[node]["weight"]*100,2)[0][0]
    
    dict_["ds"] = data_consumed["ds"].tolist()[-1]  

    return dict_ , G_final

In [ ]:
import pandas as pd
data= pd.read_feather("/Users/fablab/Renergetic/app/pages_app/data_graph_app.feather")

In [ ]:
data_pilot = data[data.pilot == "Virtual"]

In [ ]:
dict_, G_final = CalculateRenewabilityScore(data_pilot, g)

In [ ]:
data_pilot_test = data_pilot[(data_pilot.ds > "2023-05-24") & (data_pilot.domain == "electricity") & (data_pilot.type != "forecasting")].reset_index()

In [ ]:
data_pilot_test

,index,asset_name,value,ds,type,measurement_type,direction,domain,id_sensor,interpolation,pilot
0,2023747,altair,1.167,2023-05-24 00:00:00,simulated,None,in,electricity,None,None,Virtual
1,2023748,altair,1.150,2023-05-24 00:15:00,simulated,None,in,electricity,None,None,Virtual
2,2023749,altair,1.140,2023-05-24 00:30:00,simulated,None,in,electricity,None,None,Virtual
3,2023750,altair,1.137,2023-05-24 00:45:00,simulated,None,in,electricity,None,None,Virtual
4,2023751,altair,1.138,2023-05-24 01:00:00,simulated,None,in,electricity,None,None,Virtual
...,...,...,...,...,...,...,...,...,...,...,...
1339,2025374,wind_farm_1,2.368,2023-05-24 22:45:00,simulated,non_renewable,out,electricity,None,None,Virtual
1340,2025375,wind_farm_1,2.356,2023-05-24 23:00:00,simulated,non_renewable,out,electricity,None,None,Virtual
1341,2025376,wind_farm_1,2.346,2023-05-24 23:15:00,simulated,non_renewable,out,electricity,None,None,Virtual
1342,2025377,wind_farm_1,2.337,2023-05-24 23:30:00,simulated,non_renewable,out,electricity,None,None,Virtual


In [ ]:

list_dicts = []
from tqdm import tqdm
for ds_unique in tqdm(pd.unique(data_pilot_test.ds)):
    data_ds = data_pilot_test[data_pilot_test.ds == ds_unique]
    a,b = CalculateRenewabilityScore(data_ds, g)
    list_dicts.append(a)

100%|██████████| 96/96 [00:00<00:00, 197.14it/s]


In [ ]:
list_dicts

[{'building1': 38.38, 'building2': 49.12, 'ds': '2023-05-24 00:00:00'},
 {'building1': 38.43, 'building2': 49.77, 'ds': '2023-05-24 00:15:00'},
 {'building1': 38.49, 'building2': 50.07, 'ds': '2023-05-24 00:30:00'},
 {'building1': 38.48, 'building2': 50.14, 'ds': '2023-05-24 00:45:00'},
 {'building1': 38.42, 'building2': 50.03, 'ds': '2023-05-24 01:00:00'},
 {'building1': 38.32, 'building2': 49.79, 'ds': '2023-05-24 01:15:00'},
 {'building1': 38.21, 'building2': 49.49, 'ds': '2023-05-24 01:30:00'},
 {'building1': 38.1, 'building2': 49.16, 'ds': '2023-05-24 01:45:00'},
 {'building1': 37.99, 'building2': 48.88, 'ds': '2023-05-24 02:00:00'},
 {'building1': 37.93, 'building2': 48.66, 'ds': '2023-05-24 02:15:00'},
 {'building1': 37.87, 'building2': 48.56, 'ds': '2023-05-24 02:30:00'},
 {'building1': 37.86, 'building2': 48.56, 'ds': '2023-05-24 02:45:00'},
 {'building1': 37.88, 'building2': 48.65, 'ds': '2023-05-24 03:00:00'},
 {'building1': 37.92, 'building2': 48.84, 'ds': '2023-05-24 03:15